# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [ ]:
# YOUR CHANGES HERE

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import Google Drive
from google.colab import drive
drive.mount('/content/drive')


# Import twizzleflu-rewards.tsv and twizzleflu-transitions.tsv
rewards = pd.read_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/twizzleflu-rewards.tsv', sep='\t')
transitions = pd.read_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/twizzleflu-transitions.tsv', sep='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rewards.head()

,action,state,reward
0,do-nothing,exposed-1,0.0
1,do-nothing,exposed-2,0.0
2,do-nothing,exposed-3,0.0
3,do-nothing,symptoms-1,-0.5
4,do-nothing,symptoms-2,-1.0


In [ ]:
transitions.head()

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.8
1,do-nothing,exposed-1,recovered,0.2
2,do-nothing,exposed-2,exposed-3,0.8
3,do-nothing,exposed-2,recovered,0.2
4,do-nothing,exposed-3,symptoms-1,0.8


In [ ]:
# one step state-action values from a value estimate.
# will use this a lot!

def compute_qT_once(R, P, gamma, v):
    # Rewards + gamma x expected future rewards for each action
        # R means immediate rewards
        # gamma means discount factor
        # P means transition probabilities
        # v means value function
        # R has shape (nA, nS)
        # P has shape (nA, nS, nS)
        # v has shape (nS,)
        # returns Q^T with shape (nA, nS)
    return R + gamma * P @ v


def iterate_values_once(R, P, gamma, v):
    # one step of value iteration
    # returns v_{i+1} given v_i
    # this represents the maximum reward over all actions from computing Q^T
    return np.max(compute_qT_once(R, P, gamma, v), axis=0)

def value_iteration(R, P, gamma, max_iterations=100, tolerance=0.001):
    # initial approximation v_0
        # Vector of all zeros
    v_old = np.zeros(R.shape[-1])

    # Iterate up to max_iterations (default 100)
    for i in range(max_iterations):
        # compute v_{i+1}
            # one step of value iteration
        v_new = iterate_values_once(R, P, gamma, v_old)

        # check if values did not change much (less than tolerance, default 0.001)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            # If there was no significant change, return v_{i+1}
                # Only return if converged (meaning that values did not change much)
            return v_new
        # If there was significant change, continue to next iteration
            # Reassign v_{i+1} to v_i
        v_old = v_new

    # return v_{max_iterations} if not converged
    return v_old

In [ ]:
# one step state-action values from a value estimate.
# will use this a lot!

def compute_qT_once_discomfort(R, P, gamma, v):
    # (C | -R) + gamma x expected future rewards for each action
        # C | -R means immediate discomfort
        # gamma means discount factor
        # P means transition probabilities
        # v means value function
        # R has shape (nA, nS)
        # P has shape (nA, nS, nS)
        # v has shape (nS,)
        # returns Q^T with shape (nA, nS)
    return -R + gamma * P @ v

def iterate_values_once_discomfort(R, P, gamma, v):
    # one step of value iteration
    # returns v_{i+1} given v_i
    # this represents the maximum reward over all actions from computing Q^T
    return np.min(compute_qT_once_discomfort(R, P, gamma, v), axis=0)

def value_iteration_discomfort(R, P, gamma, max_iterations=100, tolerance=0.001):
    # initial approximation v_0
        # Vector of all zeros
    v_old = np.zeros(R.shape[-1], dtype=float)

    # Iterate up to max_iterations (default 100)
    for i in range(max_iterations):
        # compute v_{i+1}
            # one step of value iteration
        v_new = iterate_values_once_discomfort(R, P, gamma, v_old)
        # check if values did not change much (less than tolerance, default 0.001)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            # If there was no significant change, return v_{i+1}
                # Only return if converged (meaning that values did not change much)
            return v_new
        # If there was significant change, continue to next iteration
            # Reassign v_{i+1} to v_i
        v_old = v_new

    # return v_{max_iterations} if not converged
    return v_old

In [ ]:
a_dn = "do-nothing"

# state index
states = sorted(set(rewards["state"]) | set(transitions["state"]) | set(transitions["next_state"]))
S = {s: i for i, s in enumerate(states)}
ns = len(states)
gamma = 1.0

R = np.zeros((1, ns), dtype=float)
P = np.zeros((1, ns, ns), dtype=float)

In [ ]:
R

array([[0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
P

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
rewards

,action,state,reward
0,do-nothing,exposed-1,0.000
1,do-nothing,exposed-2,0.000
2,do-nothing,exposed-3,0.000
3,do-nothing,symptoms-1,-0.500
4,do-nothing,symptoms-2,-1.000
5,do-nothing,symptoms-3,-0.500
6,do-nothing,recovered,0.000
7,drink-oj,exposed-1,0.000
8,drink-oj,exposed-2,0.000
9,drink-oj,exposed-3,0.000


In [ ]:
# Fill R for do-nothing (one reward per state; average duplicates if any)
r_dn = (
    rewards[rewards["action"] == a_dn]
      .groupby("state", as_index=True)["reward"]
      .mean()
)
r_dn

,reward
state,
exposed-1,0.0
exposed-2,0.0
exposed-3,0.0
recovered,0.0
symptoms-1,-0.5
symptoms-2,-1.0
symptoms-3,-0.5


In [ ]:
# Fill R
for s_name, r in r_dn.items():
    if s_name in S:
        R[0, S[s_name]] = float(r)
R

array([[ 0. ,  0. ,  0. ,  0. , -0.5, -1. , -0.5]])

In [ ]:
# Fill P for do-nothing
    # Select only do-nothing transitions
subP = transitions[transitions["action"] == a_dn]
subP

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.8
1,do-nothing,exposed-1,recovered,0.2
2,do-nothing,exposed-2,exposed-3,0.8
3,do-nothing,exposed-2,recovered,0.2
4,do-nothing,exposed-3,symptoms-1,0.8
5,do-nothing,exposed-3,recovered,0.2
6,do-nothing,symptoms-1,symptoms-1,0.7
7,do-nothing,symptoms-1,symptoms-2,0.3
8,do-nothing,symptoms-2,symptoms-2,0.7
9,do-nothing,symptoms-2,symptoms-3,0.3


In [ ]:
# Sum probabilities for duplicate pairs
subP_agg = (
    subP
      .groupby(["state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP_agg

,state,next_state,probability
0,exposed-1,exposed-2,0.8
1,exposed-1,recovered,0.2
2,exposed-2,exposed-3,0.8
3,exposed-2,recovered,0.2
4,exposed-3,recovered,0.2
5,exposed-3,symptoms-1,0.8
6,recovered,recovered,1.0
7,symptoms-1,symptoms-1,0.7
8,symptoms-1,symptoms-2,0.3
9,symptoms-2,symptoms-2,0.7


In [ ]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for s_name in states:
    # Get state index
    s_idx = S[s_name]
    # Get outgoing transition rows
    rows = subP_agg[subP_agg["state"] == s_name]

    # Normalize probabilities
    if len(rows) == 0:
        # No outgoing transitions specified: stay in place
        P[0, s_idx, s_idx] = 1.0
        continue

    # Normalize probabilities by dividing by total
    total = rows["probability"].sum()
    if total <= 0:
        # Degenerate row: also make it a self-loop
        P[0, s_idx, s_idx] = 1.0
        continue

    # Fill in normalized probabilities
    for _, row in rows.iterrows():
        sp_name = row["next_state"]
        if sp_name in S:
            P[0, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [ ]:
P

array([[[0. , 0.8, 0. , 0.2, 0. , 0. , 0. ],
        [0. , 0. , 0.8, 0.2, 0. , 0. , 0. ],
        [0. , 0. , 0. , 0.2, 0.8, 0. , 0. ],
        [0. , 0. , 0. , 1. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0.7, 0.3, 0. ],
        [0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
        [0. , 0. , 0. , 0.3, 0. , 0. , 0.7]]])

In [ ]:
v_optimal_discomfort = value_iteration_discomfort(R, P, gamma)
print("Optimal value function (expected discomfort) for gamma=1.0:")
print(v_optimal_discomfort)
print()

# Pack as dictionary mapping state to value
state_values_discomfort = {states[i]: v for i, v in enumerate(v_optimal_discomfort)}
print("State values (expected discomfort) for gamma=1.0:")
print(state_values_discomfort)

# Convert directly to a dataframe
do_nothing_discomfort = pd.DataFrame(list(state_values_discomfort.items()), columns=['state', 'expected_discomfort'])
do_nothing_discomfort

Optimal value function (expected discomfort) for gamma=1.0:
[3.41097781 4.26449121 5.33132703 0.         6.66481885 4.99977911
 1.66665378]

State values (expected discomfort) for gamma=1.0:
{'exposed-1': np.float64(3.410977809744379), 'exposed-2': np.float64(4.264491212473884), 'exposed-3': np.float64(5.331327031522923), 'recovered': np.float64(0.0), 'symptoms-1': np.float64(6.664818853984066), 'symptoms-2': np.float64(4.99977911446515), 'symptoms-3': np.float64(1.6666537816771334)}


,state,expected_discomfort
0,exposed-1,3.410978
1,exposed-2,4.264491
2,exposed-3,5.331327
3,recovered,0.000000
4,symptoms-1,6.664819
5,symptoms-2,4.999779
6,symptoms-3,1.666654


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [ ]:
# YOUR CHANGES HERE

do_nothing_discomfort.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/do-nothing-discomfort.tsv', sep='\t', index=False)

Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [ ]:
# one step state-action values from a value estimate.
# will use this a lot!

def compute_qT_once(R, P, gamma, v):
    return R + gamma * P @ v

def iterate_values_once(R, P, gamma, v):
    return np.max(compute_qT_once(R, P, gamma, v), axis=0)

def value_iteration(R, P, gamma, max_iterations=100, tolerance=0.001):
    # initial approximation v_0 for each action and reward
    v_old = np.zeros(R.shape[-1], dtype=float)

    # Iterate for each action
    for a in range(R.shape[0]):
        # Iterate for each iteration
        for i in range(max_iterations):
            # compute v_{i+1} for each action
            v_new = iterate_values_once(R[a:a+1, :], P[a:a+1, :, :], gamma, v_old)
            # check if values did not change much (less than tolerance, default 0.001)
            if np.max(np.abs(v_new - v_old)) < tolerance:
                # If there was no significant change, return v_{i+1}
                return v_new
            # If there was significant change, continue to next iteration
            # Reassign v_{i+1} to v_i
            v_old = v_new

    # return v_{max_iterations}
    return v_old

In [ ]:
S, ns, gamma

({'exposed-1': 0,
  'exposed-2': 1,
  'exposed-3': 2,
  'recovered': 3,
  'symptoms-1': 4,
  'symptoms-2': 5,
  'symptoms-3': 6},
 7,
 1.0)

In [ ]:
R2 = np.zeros((3, ns), dtype=float)
P2 = np.zeros((3, ns, ns), dtype=float)

In [ ]:
R2

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
P2

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
r_all = (
    rewards
      .groupby(["action", "state"], as_index=False)["reward"]
      .mean()
)
r_all

,action,state,reward
0,do-nothing,exposed-1,0.000
1,do-nothing,exposed-2,0.000
2,do-nothing,exposed-3,0.000
3,do-nothing,recovered,0.000
4,do-nothing,symptoms-1,-0.500
5,do-nothing,symptoms-2,-1.000
6,do-nothing,symptoms-3,-0.500
7,drink-oj,exposed-1,0.000
8,drink-oj,exposed-2,0.000
9,drink-oj,exposed-3,0.000


In [ ]:
# Fill R2 for all actions (one reward per state; average duplicates if any)
for _, row in r_all.iterrows():
    a_name = row["action"]
    s_name = row["state"]
    r = row["reward"]

    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    if s_name in S:
        R2[a_idx, S[s_name]] = float(r)
R2

array([[ 0.   ,  0.   ,  0.   ,  0.   , -0.5  , -1.   , -0.5  ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.375, -0.75 , -0.375],
       [ 0.   ,  0.   ,  0.   ,  0.   , -1.   , -2.   , -1.   ]])

In [ ]:
# Fill P2 for all actions
    # For each action, select only relevant transitions
subP2 = transitions
subP2

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,symptoms-1,0.80
5,do-nothing,exposed-3,recovered,0.20
6,do-nothing,symptoms-1,symptoms-1,0.70
7,do-nothing,symptoms-1,symptoms-2,0.30
8,do-nothing,symptoms-2,symptoms-2,0.70
9,do-nothing,symptoms-2,symptoms-3,0.30


In [ ]:
# Sum probabilities for duplicate pairs
subP2_agg = (
    subP2
      .groupby(["action", "state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP2_agg

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,recovered,0.20
5,do-nothing,exposed-3,symptoms-1,0.80
6,do-nothing,recovered,recovered,1.00
7,do-nothing,symptoms-1,symptoms-1,0.70
8,do-nothing,symptoms-1,symptoms-2,0.30
9,do-nothing,symptoms-2,symptoms-2,0.70


In [ ]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for a_name in ["do-nothing", "drink-oj", "sleep-8"]:
    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    for s_name in states:
        # Get state index
        s_idx = S[s_name]
        # Get outgoing transition rows
        rows = subP2_agg[(subP2_agg["action"] == a_name) & (subP2_agg["state"] == s_name)]

        # Normalize probabilities
        if len(rows) == 0:
            # No outgoing transitions specified: stay in place
            P2[a_idx, s_idx, s_idx] = 1.0
            continue

        # Normalize probabilities by dividing by total
        total = rows["probability"].sum()
        if total <= 0:
            # Degenerate row: also make it a self-loop
            P2[a_idx, s_idx, s_idx] = 1.0
            continue

        # Fill in normalized probabilities
        for _, row in rows.iterrows():
            sp_name = row["next_state"]
            if sp_name in S:
                P2[a_idx, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [ ]:
P2

array([[[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.7 ]],

       [[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25],
        [0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.75]],

       [[0.  , 0.5 , 0.  , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.8 , 0.2 , 0.  ],
        

In [ ]:
v_optimal_reward = value_iteration(R2, P2, gamma)
print("Optimal value function (expected reward) for gamma=1.0:")
print(v_optimal_reward)
print()

# Calculate Q-values using the optimal value function
Q_optimal = compute_qT_once(R2, P2, gamma, v_optimal_reward)

# Determine the optimal policy
optimal_actions_indices = np.argmax(Q_optimal, axis=0)

# Map action indices back to action names
action_names = ["do-nothing", "drink-oj", "sleep-8"]
optimal_actions = [action_names[i] for i in optimal_actions_indices]

# Print the optimal policy
print("Optimal policy (action for each state):")
for i, state in enumerate(states):
    print(f"State: {state}, Optimal Action: {optimal_actions[i]}")

Optimal value function (expected reward) for gamma=1.0:
[-3.41097781 -4.26449121 -5.33132703  0.         -6.66481885 -4.99977911
 -1.66665378]

Optimal policy (action for each state):
State: exposed-1, Optimal Action: sleep-8
State: exposed-2, Optimal Action: sleep-8
State: exposed-3, Optimal Action: sleep-8
State: recovered, Optimal Action: do-nothing
State: symptoms-1, Optimal Action: drink-oj
State: symptoms-2, Optimal Action: drink-oj
State: symptoms-3, Optimal Action: drink-oj


In [ ]:
# Create a DataFrame of optimal actions and their expected rewards for each state
minimum_discomfort_actions = pd.DataFrame({'state': states, 'action': optimal_actions, 'expected_reward': v_optimal_reward})

print("Optimal policy (action and expected reward for each state):")
display(minimum_discomfort_actions)

Optimal policy (action and expected reward for each state):


,state,action,expected_reward
0,exposed-1,sleep-8,-3.410978
1,exposed-2,sleep-8,-4.264491
2,exposed-3,sleep-8,-5.331327
3,recovered,do-nothing,0.000000
4,symptoms-1,drink-oj,-6.664819
5,symptoms-2,drink-oj,-4.999779
6,symptoms-3,drink-oj,-1.666654


Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [ ]:
# YOUR CHANGES HERE

minimum_discomfort_actions.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/minimum-discomfort-actions.tsv', sep='\t', index=False)

Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [ ]:
R3 = np.zeros((3, ns), dtype=float)
P3 = np.zeros((3, ns, ns), dtype=float)

In [ ]:
R3

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
P3

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
# Fill R3 for all actions (one reward per state; average duplicates if any)
for _, row in r_all.iterrows():
    a_name = row["action"]
    s_name = row["state"]
    r = row["reward"]

    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    if s_name in S:
        R3[a_idx, S[s_name]] = float(r)
R3

array([[ 0.   ,  0.   ,  0.   ,  0.   , -0.5  , -1.   , -0.5  ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.375, -0.75 , -0.375],
       [ 0.   ,  0.   ,  0.   ,  0.   , -1.   , -2.   , -1.   ]])

In [ ]:
# Fill P3 for all actions
    # For each action, select only relevant transitions
subP3 = transitions
subP3

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,symptoms-1,0.80
5,do-nothing,exposed-3,recovered,0.20
6,do-nothing,symptoms-1,symptoms-1,0.70
7,do-nothing,symptoms-1,symptoms-2,0.30
8,do-nothing,symptoms-2,symptoms-2,0.70
9,do-nothing,symptoms-2,symptoms-3,0.30


In [ ]:
# Sum probabilities for duplicate pairs
subP3_agg = (
    subP3
      .groupby(["action", "state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP3_agg

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,recovered,0.20
5,do-nothing,exposed-3,symptoms-1,0.80
6,do-nothing,recovered,recovered,1.00
7,do-nothing,symptoms-1,symptoms-1,0.70
8,do-nothing,symptoms-1,symptoms-2,0.30
9,do-nothing,symptoms-2,symptoms-2,0.70


In [ ]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for a_name in ["do-nothing", "drink-oj", "sleep-8"]:
    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    for s_name in states:
        # Get state index
        s_idx = S[s_name]
        # Get outgoing transition rows
        rows = subP3_agg[(subP3_agg["action"] == a_name) & (subP3_agg["state"] == s_name)]

        # Normalize probabilities
        if len(rows) == 0:
            # No outgoing transitions specified: stay in place
            P3[a_idx, s_idx, s_idx] = 1.0
            continue

        # Normalize probabilities by dividing by total
        total = rows["probability"].sum()
        if total <= 0:
            # Degenerate row: also make it a self-loop
            P3[a_idx, s_idx, s_idx] = 1.0
            continue

        # Fill in normalized probabilities
        for _, row in rows.iterrows():
            sp_name = row["next_state"]
            if sp_name in S:
                P3[a_idx, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [ ]:
P3

array([[[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.7 ]],

       [[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25],
        [0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.75]],

       [[0.  , 0.5 , 0.  , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.8 , 0.2 , 0.  ],
        

In [ ]:
# Compute the expected discomfort for the optimal policy using policy evaluation (solving linear system)

# Get the optimal policy from the DataFrame
optimal_policy = minimum_discomfort_actions.set_index('state')['action']

# Construct the reward vector r_pi for the optimal policy
  # Use discomfort rewards (-R2)
c_pi = np.zeros(ns)
for i, state in enumerate(states):
    action = optimal_policy[state]
    if action == "do-nothing":
        a_idx = 0
    elif action == "drink-oj":
        a_idx = 1
    elif action == "sleep-8":
        a_idx = 2
    else:
        continue
    # Immediate discomfort for the optimal action in each state
    c_pi[i] = -R2[a_idx, i]

# Construct the transition matrix P_pi for the optimal policy
P3 = np.zeros((ns, ns))
for i, state in enumerate(states):
    action = optimal_policy[state]
    if action == "do-nothing":
        a_idx = 0
    elif action == "drink-oj":
        a_idx = 1
    elif action == "sleep-8":
        a_idx = 2
    else:
        continue
    # Transition probabilities for the optimal action in each state
    P3[i, :] = P2[a_idx, i, :]

# Solve the linear system (I - P_pi) * d_pi = c_pi for d_pi
  # Where d_pi is the expected discomfort for each state under the policy
P3_identity_matrix = np.eye(ns)
P3_inverted_matrix = P3_identity_matrix - P3

# Handle the recovered state
recovered_idx = S['recovered']
indices_to_solve = [i for i in range(ns) if i != recovered_idx]

reduced_matrix = P3_inverted_matrix[np.ix_(indices_to_solve, indices_to_solve)]
reduced_c_pi = c_pi[indices_to_solve]

# Solve the reduced system
d_pi_reduced = np.linalg.solve(reduced_matrix, reduced_c_pi)

# Reconstruct the full discomfort vector, placing 0 for the recovered state
d_pi = np.zeros(ns)
for i, original_idx in enumerate(indices_to_solve):
    d_pi[original_idx] = d_pi_reduced[i]

# The discomfort for the recovered state is 0
d_pi[recovered_idx] = 0.0

print("Expected discomfort for each state under the optimal policy (solved via linear system):")
print(d_pi)
print()

# Pack as dictionary mapping state to value
state_values_discomfort = {states[i]: v for i, v in enumerate(d_pi)}
print("State values (expected discomfort) for gamma=1.0:")
print(state_values_discomfort)

# Convert directly to a dataframe
minimum_discomfort_values = pd.DataFrame(list(state_values_discomfort.items()), columns=['state', 'expected_discomfort'])
display(minimum_discomfort_values)

Expected discomfort for each state under the optimal policy (solved via linear system):
[0.75 1.5  3.   0.   6.   4.5  1.5 ]

State values (expected discomfort) for gamma=1.0:
{'exposed-1': np.float64(0.75), 'exposed-2': np.float64(1.5), 'exposed-3': np.float64(3.0), 'recovered': np.float64(0.0), 'symptoms-1': np.float64(6.0), 'symptoms-2': np.float64(4.5), 'symptoms-3': np.float64(1.5)}


,state,expected_discomfort
0,exposed-1,0.75
1,exposed-2,1.50
2,exposed-3,3.00
3,recovered,0.00
4,symptoms-1,6.00
5,symptoms-2,4.50
6,symptoms-3,1.50


Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [ ]:
# YOUR CHANGES HERE

minimum_discomfort_values.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/minimum-discomfort-values.tsv', sep='\t', index=False)

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [ ]:
rewards

,action,state,reward
0,do-nothing,exposed-1,0.000
1,do-nothing,exposed-2,0.000
2,do-nothing,exposed-3,0.000
3,do-nothing,symptoms-1,-0.500
4,do-nothing,symptoms-2,-1.000
5,do-nothing,symptoms-3,-0.500
6,do-nothing,recovered,0.000
7,drink-oj,exposed-1,0.000
8,drink-oj,exposed-2,0.000
9,drink-oj,exposed-3,0.000


In [ ]:
sick_states = rewards["state"].unique()

# Drop "recovered"
sick_states = sick_states[sick_states != "recovered"]
sick_states

array(['exposed-1', 'exposed-2', 'exposed-3', 'symptoms-1', 'symptoms-2',
       'symptoms-3'], dtype=object)

In [ ]:
states = rewards["state"].unique()
states

array(['exposed-1', 'exposed-2', 'exposed-3', 'symptoms-1', 'symptoms-2',
       'symptoms-3', 'recovered'], dtype=object)

In [ ]:
# YOUR CHANGES HERE
# Duplicate the rewards as duration_rewards
duration_rewards = rewards.copy()

# For each sick state (in the state column), change rewards to -1, otherwise, change to 0
for state in states:
  if state in sick_states:
    duration_rewards.loc[duration_rewards['state'] == state, 'reward'] = -1
  else:
    duration_rewards.loc[duration_rewards['state'] == state, 'reward'] = 0

In [ ]:
# Convert duration_rewards to a dataframe
duration_rewards = pd.DataFrame(duration_rewards)
duration_rewards

,action,state,reward
0,do-nothing,exposed-1,-1.0
1,do-nothing,exposed-2,-1.0
2,do-nothing,exposed-3,-1.0
3,do-nothing,symptoms-1,-1.0
4,do-nothing,symptoms-2,-1.0
5,do-nothing,symptoms-3,-1.0
6,do-nothing,recovered,0.0
7,drink-oj,exposed-1,-1.0
8,drink-oj,exposed-2,-1.0
9,drink-oj,exposed-3,-1.0


Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [ ]:
# YOUR CHANGES HERE

duration_rewards.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/duration-rewards.tsv', sep='\t', index=False)

Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [ ]:
# # YOUR CHANGES HERE

# # Modify for the situation
# def compute_qT_once_conditional(R, P, gamma, v):
#     # If the current state is not "recovered", the reward is always -1
#     if v[S] != "recovered":
#         return -1
#     # Otherwise, the reward is always 0
#     else:
#         return 0

# def iterate_values_once_conditional(R, P, gamma, v):
#     return np.max(compute_qT_once_conditional(R, P, gamma, v), axis=0)

# def value_iteration_conditional(R, P, gamma, max_iterations=100, tolerance=0.001):
#     # initial approximation v_0 for each action and reward
#     v_old = np.zeros(R.shape[-1], dtype=float)

#     # Iterate for each action
#     for a in range(R.shape[0]):
#         # Iterate for each iteration
#         for i in range(max_iterations):
#             # compute v_{i+1} for each action
#             v_new = iterate_values_once(R[a:a+1, :], P[a:a+1, :, :], gamma, v_old)
#             # check if values did not change much (less than tolerance, default 0.001)
#             if np.max(np.abs(v_new - v_old)) < tolerance:
#                 # If there was no significant change, return v_{i+1}
#                 return v_new
#             # If there was significant change, continue to next iteration
#             # Reassign v_{i+1} to v_i
#             v_old = v_new

#     # return v_{max_iterations}
#     return v_old

In [ ]:
# Actions do not matter
R5 = np.zeros((3, ns), dtype=float)
P5 = np.zeros((3, ns, ns), dtype=float)

In [ ]:
R5

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
P5

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
# Fill R4 (one reward per state; average duplicates if any)
r5_all = (
    duration_rewards
      .groupby(["action", "state"], as_index=False)["reward"]
      .mean()
)
r5_all

,action,state,reward
0,do-nothing,exposed-1,-1.0
1,do-nothing,exposed-2,-1.0
2,do-nothing,exposed-3,-1.0
3,do-nothing,recovered,0.0
4,do-nothing,symptoms-1,-1.0
5,do-nothing,symptoms-2,-1.0
6,do-nothing,symptoms-3,-1.0
7,drink-oj,exposed-1,-1.0
8,drink-oj,exposed-2,-1.0
9,drink-oj,exposed-3,-1.0


In [ ]:
# Fill R4 for all actions (one reward per state; average duplicates if any)
for _, row in r5_all.iterrows():
    a_name = row["action"]
    s_name = row["state"]
    r = row["reward"]

    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    if s_name in S:
        R5[a_idx, S[s_name]] = float(r)
R5

array([[-1., -1., -1.,  0., -1., -1., -1.],
       [-1., -1., -1.,  0., -1., -1., -1.],
       [-1., -1., -1.,  0., -1., -1., -1.]])

In [ ]:
# Fill P5
subP5 = transitions
subP5

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,symptoms-1,0.80
5,do-nothing,exposed-3,recovered,0.20
6,do-nothing,symptoms-1,symptoms-1,0.70
7,do-nothing,symptoms-1,symptoms-2,0.30
8,do-nothing,symptoms-2,symptoms-2,0.70
9,do-nothing,symptoms-2,symptoms-3,0.30


In [ ]:
subP5_agg = (
    subP5
      .groupby(["action", "state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP5_agg

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,recovered,0.20
5,do-nothing,exposed-3,symptoms-1,0.80
6,do-nothing,recovered,recovered,1.00
7,do-nothing,symptoms-1,symptoms-1,0.70
8,do-nothing,symptoms-1,symptoms-2,0.30
9,do-nothing,symptoms-2,symptoms-2,0.70


In [ ]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for a_name in ["do-nothing", "drink-oj", "sleep-8"]:
    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    for s_name in states:
        # Get state index
        s_idx = S[s_name]
        # Get outgoing transition rows
        rows = subP5_agg[(subP5_agg["action"] == a_name) & (subP5_agg["state"] == s_name)]

        # Normalize probabilities
        if len(rows) == 0:
            # No outgoing transitions specified: stay in place
            P5[a_idx, s_idx, s_idx] = 1.0
            continue

        # Normalize probabilities by dividing by total
        total = rows["probability"].sum()
        if total <= 0:
            # Degenerate row: also make it a self-loop
            P5[a_idx, s_idx, s_idx] = 1.0
            continue

        # Fill in normalized probabilities
        for _, row in rows.iterrows():
            sp_name = row["next_state"]
            if sp_name in S:
                P5[a_idx, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [ ]:
P5

array([[[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.7 ]],

       [[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25],
        [0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.75]],

       [[0.  , 0.5 , 0.  , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.8 , 0.2 , 0.  ],
        

In [ ]:
v_optimal_duration_reward = value_iteration(R5, P5, gamma)
print("Optimal value function (expected duration reward) for gamma=1.0:")
print(v_optimal_duration_reward)
print()

# Calculate Q-values using the optimal value function
Q_duration_optimal = compute_qT_once(R5, P5, gamma, v_optimal_duration_reward)

# Determine the optimal policy
optimal_duration_actions_indices = np.argmax(Q_duration_optimal, axis=0)

# Map action indices back to action names
duration_action_names = ["do-nothing", "drink-oj", "sleep-8"]
optimal_duration_actions = [duration_action_names[i] for i in optimal_duration_actions_indices]

# Print the optimal policy
print("Optimal policy (action for each state) for minimizing duration:")
for i, state in enumerate(states):
    print(f"State: {state}, Optimal Action: {optimal_duration_actions[i]}")

Optimal value function (expected duration reward) for gamma=1.0:
[-7.55770928 -8.19788455 -8.99804914  0.         -9.99820332 -6.666452
 -3.33332071]

Optimal policy (action for each state) for minimizing duration:
State: exposed-1, Optimal Action: sleep-8
State: exposed-2, Optimal Action: sleep-8
State: exposed-3, Optimal Action: sleep-8
State: symptoms-1, Optimal Action: do-nothing
State: symptoms-2, Optimal Action: do-nothing
State: symptoms-3, Optimal Action: do-nothing
State: recovered, Optimal Action: do-nothing


In [ ]:
# Create a DataFrame of optimal actions and their expected duration for each state
# The expected duration is the negative of the optimal value function for minimizing duration
minimum_duration = pd.DataFrame({'state': states, 'action': optimal_duration_actions, 'expected_duration': -v_optimal_duration_reward})

print("Optimal policy (action and expected duration for each state):")
display(minimum_duration)

Optimal policy (action and expected duration for each state):


,state,action,expected_duration
0,exposed-1,sleep-8,7.557709
1,exposed-2,sleep-8,8.197885
2,exposed-3,sleep-8,8.998049
3,symptoms-1,do-nothing,-0.000000
4,symptoms-2,do-nothing,9.998203
5,symptoms-3,do-nothing,6.666452
6,recovered,do-nothing,3.333321


In [ ]:
# Only keep the state and action columns in minimum_duration_actions
minimum_duration_actions = minimum_duration[['state', 'action']]
minimum_duration_actions

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,symptoms-1,do-nothing
4,symptoms-2,do-nothing
5,symptoms-3,do-nothing
6,recovered,do-nothing


Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [ ]:
# YOUR CHANGES HERE

minimum_duration_actions.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/minimum-duration-actions.tsv', sep='\t', index=False)

Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [ ]:
# YOUR CHANGES HERE

# Create a DataFrame of optimal states and expected_sick_days
minimum_duration_days = pd.DataFrame({'state': states, 'expected_sick_days': -v_optimal_duration_reward})

print("Optimal policy (expected sick days for each state):")
display(minimum_duration_days)

Optimal policy (expected sick days for each state):


,state,expected_sick_days
0,exposed-1,7.557709
1,exposed-2,8.197885
2,exposed-3,8.998049
3,symptoms-1,-0.000000
4,symptoms-2,9.998203
5,symptoms-3,6.666452
6,recovered,3.333321


Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [ ]:
# YOUR CHANGES HERE

minimum_duration_days.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/minimum-duration-days.tsv', sep='\t', index=False)

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [ ]:
# YOUR CHANGES HERE

# Implement minimum_duration_actions to compute the expected discomfort
R7 = np.zeros((3, ns), dtype=float)
P7 = np.zeros((3, ns, ns), dtype=float)

In [ ]:
R7

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
P7

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
# Fill R7 for all actions (one reward per state; average duplicates if any)
for _, row in r_all.iterrows():
    a_name = row["action"]
    s_name = row["state"]
    r = row["reward"]

    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    if s_name in S:
        R7[a_idx, S[s_name]] = float(r)
R7

array([[ 0.   ,  0.   ,  0.   ,  0.   , -0.5  , -1.   , -0.5  ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.375, -0.75 , -0.375],
       [ 0.   ,  0.   ,  0.   ,  0.   , -1.   , -2.   , -1.   ]])

In [ ]:
# Fill P7 for all actions
    # For each action, select only relevant transitions
subP7 = transitions
subP7

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,symptoms-1,0.80
5,do-nothing,exposed-3,recovered,0.20
6,do-nothing,symptoms-1,symptoms-1,0.70
7,do-nothing,symptoms-1,symptoms-2,0.30
8,do-nothing,symptoms-2,symptoms-2,0.70
9,do-nothing,symptoms-2,symptoms-3,0.30


In [ ]:
# Sum probabilities for duplicate pairs
subP7_agg = (
    subP7
      .groupby(["action", "state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP7_agg

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.80
1,do-nothing,exposed-1,recovered,0.20
2,do-nothing,exposed-2,exposed-3,0.80
3,do-nothing,exposed-2,recovered,0.20
4,do-nothing,exposed-3,recovered,0.20
5,do-nothing,exposed-3,symptoms-1,0.80
6,do-nothing,recovered,recovered,1.00
7,do-nothing,symptoms-1,symptoms-1,0.70
8,do-nothing,symptoms-1,symptoms-2,0.30
9,do-nothing,symptoms-2,symptoms-2,0.70


In [ ]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for a_name in ["do-nothing", "drink-oj", "sleep-8"]:
    if a_name == "do-nothing":
        a_idx = 0
    elif a_name == "drink-oj":
        a_idx = 1
    elif a_name == "sleep-8":
        a_idx = 2
    else:
        continue

    for s_name in states:
        # Get state index
        s_idx = S[s_name]
        # Get outgoing transition rows
        rows = subP7_agg[(subP7_agg["action"] == a_name) & (subP7_agg["state"] == s_name)]

        # Normalize probabilities
        if len(rows) == 0:
            # No outgoing transitions specified: stay in place
            P7[a_idx, s_idx, s_idx] = 1.0
            continue

        # Normalize probabilities by dividing by total
        total = rows["probability"].sum()
        if total <= 0:
            # Degenerate row: also make it a self-loop
            P7[a_idx, s_idx, s_idx] = 1.0
            continue

        # Fill in normalized probabilities
        for _, row in rows.iterrows():
            sp_name = row["next_state"]
            if sp_name in S:
                P7[a_idx, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [ ]:
P7

array([[[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.7 ]],

       [[0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.8 , 0.2 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.2 , 0.8 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25],
        [0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.75]],

       [[0.  , 0.5 , 0.  , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.5 , 0.5 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.8 , 0.2 , 0.  ],
        

In [ ]:
# Compute the expected discomfort for the optimal policy using policy evaluation (solving linear system)

# Get the optimal policy for minimizing duration from the DataFrame
optimal_duration_policy = minimum_duration_actions.set_index('state')['action']

# Construct the reward vector r_pi for the optimal sick days policy
# Use the original discomfort rewards (-R2) for the actions in the optimal duration policy
c_pi_duration = np.zeros(ns)
for i, state in enumerate(states):
    action = optimal_duration_policy[state]
    if action == "do-nothing":
        a_idx = 0
    elif action == "drink-oj":
        a_idx = 1
    elif action == "sleep-8":
        a_idx = 2
    else:
        continue
    # Immediate discomfort for the optimal action in each state
    # Use the discomfort values from R7 (which is -R2)
    c_pi_duration[i] = -R7[a_idx, i]

# Construct the transition matrix P_pi for the optimal duration policy
P7_pi_duration = np.zeros((ns, ns))
for i, state in enumerate(states):
    action = optimal_duration_policy[state]
    if action == "do-nothing":
        a_idx = 0
    elif action == "drink-oj":
        a_idx = 1
    elif action == "sleep-8":
        a_idx = 2
    else:
        continue
    # Transition probabilities for the optimal action in each state from P4
    P7_pi_duration[i, :] = P5[a_idx, i, :]

# Solve the linear system (I - P_pi) * d_pi = c_pi for d_pi
# Where d_pi is the expected discomfort for each state under the policy
P7_pi_duration_identity_matrix = np.eye(ns)
P7_pi_duration_inverted_matrix = P7_pi_duration_identity_matrix - P7_pi_duration

# Handle the recovered state (expected discomfort is 0)
recovered_idx_P7_pi_duration = S['recovered']
indices_to_solve_P7_pi_duration = [i for i in range(ns) if i != recovered_idx_P7_pi_duration]

reduced_matrix_P7_pi_duration = P7_pi_duration_inverted_matrix[np.ix_(indices_to_solve_P7_pi_duration, indices_to_solve_P7_pi_duration)]
reduced_c_pi_duration_P7 = c_pi_duration[indices_to_solve_P7_pi_duration]

# Solve the reduced system
d_pi_reduced_P7_pi_duration = np.linalg.solve(reduced_matrix_P7_pi_duration, reduced_c_pi_duration_P7)

# Reconstruct the full discomfort vector, placing 0 for the recovered state
d_pi_P7_pi_duration = np.zeros(ns)
for i, original_idx_P7_pi_duration in enumerate(indices_to_solve_P7_pi_duration):
    d_pi_P7_pi_duration[original_idx_P7_pi_duration] = d_pi_reduced_P7_pi_duration[i]

# The discomfort for the recovered state is 0
d_pi_P7_pi_duration[recovered_idx_P7_pi_duration] = 0.0

print("Expected discomfort for each state under the optimal duration policy (solved via linear system):")
print(d_pi_P7_pi_duration)
print()

# Pack as dictionary mapping state to value
state_values_speed_discomfort = {states[i]: v for i, v in enumerate(d_pi_P7_pi_duration)}
print("State values (optimal duration) for gamma=1.0:")
print(state_values_speed_discomfort)

# Convert directly to a dataframe
speed_discomfort_values = pd.DataFrame(list(state_values_speed_discomfort.items()), columns=['state', 'speed_discomfort'])
display(speed_discomfort_values)

Expected discomfort for each state under the optimal duration policy (solved via linear system):
[0.83333333 1.66666667 3.33333333 0.         6.66666667 5.
 1.66666667]

State values (optimal duration) for gamma=1.0:
{'exposed-1': np.float64(0.8333333333333331), 'exposed-2': np.float64(1.6666666666666663), 'exposed-3': np.float64(3.3333333333333326), 'symptoms-1': np.float64(0.0), 'symptoms-2': np.float64(6.666666666666665), 'symptoms-3': np.float64(4.999999999999999), 'recovered': np.float64(1.6666666666666665)}


,state,speed_discomfort
0,exposed-1,0.833333
1,exposed-2,1.666667
2,exposed-3,3.333333
3,symptoms-1,0.000000
4,symptoms-2,6.666667
5,symptoms-3,5.000000
6,recovered,1.666667


In [ ]:
minimum_discomfort_values

,state,expected_discomfort
0,exposed-1,0.75
1,exposed-2,1.50
2,exposed-3,3.00
3,recovered,0.00
4,symptoms-1,6.00
5,symptoms-2,4.50
6,symptoms-3,1.50


In [ ]:
# Copy minimum discomfort values
minimum_discomfort_values_copy = minimum_discomfort_values.copy()

# Reorder the states based on the same order as speed discomfort values
minimum_discomfort_values_copy['state'] = pd.Categorical(minimum_discomfort_values_copy['state'], categories=speed_discomfort_values['state'], ordered=True)
minimum_discomfort_values_copy = minimum_discomfort_values_copy.sort_values('state')

# Reset the index
minimum_discomfort_values_copy = minimum_discomfort_values_copy.reset_index(drop=True)
minimum_discomfort_values_copy

,state,expected_discomfort
0,exposed-1,0.75
1,exposed-2,1.50
2,exposed-3,3.00
3,symptoms-1,6.00
4,symptoms-2,4.50
5,symptoms-3,1.50
6,recovered,0.00


In [ ]:
# Use the minimum_discomfort_values and concatenate them with the speed_discomfort_values as the minimize_discomfort values
policy_comparison = pd.concat([speed_discomfort_values, minimum_discomfort_values_copy.drop("state", axis=1)], axis=1)

# Rename "expected_discomfort" with minimize_discomfort
policy_comparison = policy_comparison.rename(columns={"expected_discomfort": "minimize_discomfort"})
policy_comparison

,state,speed_discomfort,minimize_discomfort
0,exposed-1,0.833333,0.75
1,exposed-2,1.666667,1.50
2,exposed-3,3.333333,3.00
3,symptoms-1,0.000000,6.00
4,symptoms-2,6.666667,4.50
5,symptoms-3,5.000000,1.50
6,recovered,1.666667,0.00


Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [ ]:
# YOUR CHANGES HERE

policy_comparison.to_csv('/content/drive/MyDrive/Online MSDS/MOD 8/Project_6/policy-comparison.tsv', sep='\t', index=False)

Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

Arun Ram during office hours.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

None.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

I used Gemini via Colab to explain my code to understand where fixes should be implemented based on errors that were output.